<h1> Final model</h1>
After performing analysis on several models, the best model which I decided as the best model was LinerSVM model. It has the capability to deal with huge data also yield good results.

In [2]:
#Importing Libraries
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from tqdm import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
# Import NLTK libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
#Import Pandas
import pandas as pd
# Read the final_full_feature_reviews file
data=pd.read_csv('final_full_feature_reviews.csv')
data=data[['clean_text','review_stars']]

# Cleaning the data by dropping Null value data
data['clean_text'].dropna(inplace=True)
# Use lowercace for the text in clean_text
data['clean_text']=[i.lower() for i in data['clean_text']]
# Use word_tokenize for data in clean text which is used to split large sample of text into words
data['clean_text']=[word_tokenize(i) for i in data['clean_text']]

In [5]:
# Word Lemmatizer initially understands a word as Noun. To categorize a word with POS tag, word lemmatizer 
# needs to understand whether it is an adjective or verb or adverb.
t_map = defaultdict(lambda : wn.NOUN)
t_map['J'] = wn.ADJ
t_map['V'] = wn.VERB
t_map['R'] = wn.ADV

In [6]:
for ind,ent in tqdm(enumerate(data['clean_text'])):
    # Declare a list to store the words
    word_list = []
    # Initialize WordNetLemmatizer()
    w_Lemmatized = WordNetLemmatizer()
    # pos_tag function provides the 'tag' for the words whether it is a Noun(N) or Verb(V) or something.
    for wrd, tag in pos_tag(ent):
        # consider only alphabets and check for Stop words 
        if wrd not in stopwords.words('english') and wrd.isalpha():
            w_Final = w_Lemmatized.lemmatize(wrd,t_map[tag[0]])
            word_list.append(w_Final)
       # After every iteration, list of processede words will be stored in 'text_final'
    data.loc[ind,'text_final'] = str(word_list)



100000it [44:59, 37.04it/s]


In [7]:
data.to_csv('full_data_tfidf.csv', index=False)


In [8]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(data['text_final'],data['review_stars'],test_size=0.3)

# Encode the target variable to transform Categorical data of string type into numerical values
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

# Vectorize the words by using TF-IDF Vectorizer to find how important a word in document compared to corpus
tfidf_v = TfidfVectorizer(max_features=5000)
tfidf_v.fit(data['text_final'])

Train_X_Tfidf = tfidf_v.transform(Train_X)
Test_X_Tfidf = tfidf_v.transform(Test_X)

In [9]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  65.16999999999999


In [10]:
# Download the ".pkl" file direct model execution
from sklearn.externals import joblib
joblib.dump(SVM, 'svm.pkl')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['svm.pkl']

In [11]:
# Cross tab for actual vs Predicted
pd.crosstab(Test_Y,predictions_SVM)

col_0,0,1,2,3,4
row_0,,,,,
0,3317,418,178,92,191
1,780,716,533,235,126
2,267,420,1137,1060,437
3,117,112,617,3046,2926
4,143,45,150,1602,11335


In [12]:
Train_X.shape

(70000,)

In [13]:
Train_X_Tfidf.shape

(70000, 5000)